In [1]:
import pandas as pd
import numpy as np
import re

I import the subset from ZINC15 taken from Chris' dataset on the rostserver: "canonical_smiles.txt". It conains 100 canonical SMILES.

In [2]:
subset = pd.read_table('../datasets/sets_from_Chris/canonical_smiles_first_100.txt', header=None)

subset = subset.drop_duplicates()          # throw out duplicates

In [3]:
subset = subset.rename(columns={0: "canonical_SMILES"})

- Wofür steht das @? \
In SMILES, tetrahedral centers may be indicated by a simplified chiral specification (@ or @@) written as an atomic property following the atomic symbol of the chiral atom.\
The symbol "@" indicates that the following neighbors are listed anticlockwise (it is a "visual mnemonic" in that the symbol looks like an anticlockwise spiral around a central circle).\
"@@" indicates that the neighbors are listed clockwise (you guessed it, anti-anti-clockwise). 


- Wurde die Transformation mit RDKit gemacht?

# Find out the alphabet

Are we interested in the non-alphabetic signs? I can't think of any.

In [4]:
total_SMILES = "".join(subset.canonical_SMILES)

There are lower-case letters because:
- The second letter of two-character symbols must be entered in lower case.
- Atoms in aromatic rings are specified by lower case letters, e.g., aliphatic carbon is represented by the capital letter C, aromatic carbon by lower case c.

=> I must find also 2-letter elements!

### 2-letter elements

First, extract the 2-letter elements.

In [5]:
two_letter_elements = re.findall("[A-Z][a-z]", total_SMILES)
two_letter_elements = sorted(set(two_letter_elements))

In [6]:
two_letter_elements

['Cc', 'Cl', 'Nc', 'Oc', 'Si']

In [7]:
periodic_table_2_letter = ['He',
                           'Li', 'Be', 'Ne',
                           'Na', 'Mg', 'Al', 'Si', 'Cl', 'Ar',
                           'Ca', 'Sc', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
                           'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'Xe',
                           'Cs', 'Ba', 'Hf', 'Ta', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn',
                           'Fr', 'Ra', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og',
                           'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu',
                           'Ac', 'Th', 'Pa', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr']

In [8]:
# are all extracted 2-letter elements in the periodic table?
two_letter_elements in periodic_table_2_letter

False

In [9]:
#set(two_letter_elements).intersection(set(periodic_table_2_letter))     # matches
#set(periodic_table_2_letter).symmetric_difference(set(two_letter_elements))    # elements which are
                                                                                # exclusive to either set

In [10]:
nonsensicals = [element for element in two_letter_elements if element not in periodic_table_2_letter]
nonsensicals

['Cc', 'Nc', 'Oc']

ATTENTION: At this point arises the issue of how to handle the exceptions. They should always be manually inspected.
As all currently occuring exceptions can be explained, I throw them out.

In [13]:
# throw out nonsensical 2-letter elements
two_letter_elements = [element for element in two_letter_elements if element in periodic_table_2_letter]

### 1-letter elements

In [14]:
total_SMILES = re.sub("[A-Z][a-z]", "", total_SMILES)   # remove all 2-letter elements

Now I throw out all symbols like brackets, at-signs etc.

In [16]:
only_alpha = ""

for char in total_SMILES:
    
    # CHECKING FOR UPPER CASE
    if ord(char) >= 65 and ord(char) <= 90:      # ord(chr) returns the ascii value
        only_alpha += char
        
        # checking for lower case        
    elif ord(char) >= 97 and ord(char) <= 122:
        only_alpha += char

#print(only_alpha)

In the following, I want to ignore cases, as in 'C' and 'c' only count as 'C' in the final alphabet.

In [17]:
only_alpha = only_alpha.upper()

Now I will, as above with the 2-letter elements, check, if all found elements are actually in the periodic table.

In [18]:
periodic_table_1_letter = ['H',
                           'B', 'C', 'N', 'O', 'F',
                           'P', 'S',
                           'K',
                           'Y', 'I',
                           'W',
                           'U']

In [24]:
nonsensicals = [element for element in only_alpha if element not in periodic_table_1_letter]
nonsensicals

# In case that there are nonsense-elements, analyse them here and then throw them out.

[]

In [25]:
alphabet = sorted(set(only_alpha))
alphabet = sorted(alphabet + two_letter_elements)   # Combine the element lists
alphabet

['C', 'Cl', 'F', 'H', 'N', 'O', 'P', 'S', 'Si']